In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from torch import nn 
from torchvision import models
import torch 
import gym , sys
import cv2
from tqdm import tqdm

from rlxai import A2C
env = gym.make("Breakout-v0")

/opt/conda/envs/xai/lib/python3.8/site-packages/ale_py/roms/__init__.py:94: DeprecationWarning: Automatic importing of atari-py roms won't be supported in future releases of ale-py. Please migrate over to using `ale-import-roms` OR an ALE-supported ROM package. To make this warning disappear you can run `ale-import-roms --import-from-pkg atari_py.atari_roms`.For more information see: https://github.com/mgbellemare/Arcade-Learning-Environment#rom-management
  _RESOLVED_ROMS = _resolve_roms()
A.L.E: Arcade Learning Environment (version +978d2ce)
[Powered by Stella]


In [3]:

def img_crop(img_arr) :
    return img_arr[55:-15,15:-15,:]
def rgb2gray(rgb):
    image_data = cv2.cvtColor(cv2.resize(rgb, (84, 84)), cv2.COLOR_BGR2GRAY)
    image_data[image_data > 0] = 255
    image_data = np.reshape(image_data,(84, 84, 1))
    return image_data
def totensor(img_arr) :
    return torch.FloatTensor(img_arr.transpose((2, 0, 1))).unsqueeze(dim=0)


In [4]:
def data_transform(x) :
    x = img_crop(x)
    x = rgb2gray(x)
    x = totensor(x)
    return x

In [5]:
s = env.reset()
ss = data_transform(s)
ss.shape

torch.Size([1, 1, 84, 84])

In [6]:
actor = models.resnet18(pretrained=True)
actor.conv1 =nn.Conv2d(1,64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
actor.fc = nn.Linear(512, env.action_space.n)

critic = models.resnet18(pretrained=True)
critic.conv1 =nn.Conv2d(1,64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
critic.fc = nn.Linear(512, 1)

In [7]:
def data_transform(x) :
    x = img_crop(x)
    x = rgb2gray(x)
    x = totensor(x)
    return x

In [8]:
A2C_MODEL = A2C(actor=actor,critic=critic,lr=1e-3,gamma=0.99,lam=0.9)

In [9]:
from IPython.display import clear_output
from array2gif import  write_gif 

In [12]:
############################################################
rewards      = []  #
is_terminals = []  #
values       = []  #
logprobs     = []  #
############################################################
max_steps_per_episode = 1_000_000
batch_size = 1000
s = env.reset()
A2C_MODEL.load_model("./")
img_collection = [s.transpose((2, 0, 1))]
with tqdm(total=max_steps_per_episode, file=sys.stdout) as pbar:
    for t in range(1, max_steps_per_episode):
        s_tensor = data_transform(s)
        act , v , logprob = A2C_MODEL.choose_action(s_tensor,inference=False)
        s,r,done,info = env.step(int(act))
        rewards.append(r)
        is_terminals.append(done)
        values.append(v)
        logprobs.append(logprob)
        if (batch_size ==len(rewards)) | done :
            s_tensor = data_transform(s)
            _ , v , _ = A2C_MODEL.choose_action(s_tensor,inference=False)
            actor_loss,  critic_loss = A2C_MODEL.get_loss(rewards , is_terminals,values,logprobs,float(v) )
            print()
            A2C_MODEL.train(actor_loss=actor_loss,critic_loss=critic_loss)
            A2C_MODEL.save_model("./")
            ############################################################
            rewards      = []  #
            is_terminals = []  #
            values       = []  #
            logprobs     = []  #
            ############################################################
            a_l , c_l =actor_loss.detach().numpy() , critic_loss.detach().numpy()
            
            write_gif(img_collection,f"./gif/eval_{t:05d}.gif")
            del img_collection
            img_collection = []
            if done :
                print("restart...")
                s = env.reset()
                img_collection.append(s.transpose((2, 0, 1)))
        else :
            if t % 10 == 0 :
                img_collection.append(s.transpose((2, 0, 1)))
            a_l , c_l = 0,0
        #pbar.set_description(f"{done} {r} {a_l:3f}, {c_l:3f}")
        pbar.update(1)

  0%|          | 1999/1000000 [06:48<56:37:32,  4.90it/s]


KeyboardInterrupt: 